## **Deep Learning**

In [ ]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

ValueError: ignored

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
!pip install hickle
import hickle as hkl
import os
import matplotlib.pyplot as plt
from keras import layers,Input,Sequential
from keras.layers import Dense,Flatten,Reshape,Conv2DTranspose,Conv2D,MaxPooling2D,Reshape,Resizing,Dropout
from tensorflow.keras import datasets, layers, models, losses, Model
from random import randint
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 2.3 MB/s eta 0:00:00
Mounted at /gdrive
/gdrive


In [ ]:
load_hkl = hkl.load('/gdrive/My Drive/BitirmeTezi/image_dataset.hkl')
X_train=load_hkl['xtrain'] / 255.
X_test=load_hkl['xtest'] / 255.
y_train=load_hkl['ytrain']
y_test=load_hkl['ytest']

In [ ]:
X_train.shape

(427, 200, 200, 3)

## **Burada Rescaling Layer autoencoder da olmaz çünkü xtrain verilseydi, normalize edilerek kullanılırdı ama çıkışta normalize edilmemiş xtrain ile karşılaştırılırdı !!!!!**

In [ ]:
class CNN_Autoencoder(Model):
    def __init__(self):
        super(CNN_Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=X_train.shape[1:]),
            layers.Conv2D(16, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=1),
            layers.Conv2D(32, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=1),
            layers.Conv2D(64, 3, strides=1, padding='same', activation='relu'),
            layers.MaxPooling2D(2, strides=1),
            Flatten(),
            Dense(512,activation='relu'),
            Dense(256,activation='relu'),
            Dense(150,activation='relu')])

        self.decoder = tf.keras.Sequential([
            layers.Dense(150,activation='relu'),
            layers.Dense(500,activation='relu'),
            layers.Dense(2500,activation='relu'),
            layers.Reshape((50,50,1)),
            layers.Conv2D(512, 3, strides=1, padding='same', activation='relu'),
            layers.UpSampling2D(2),
            layers.Conv2D(128, 3, strides=1, padding='same', activation='relu'),
            layers.UpSampling2D(2),
            layers.Conv2D(3, 3, strides=1, padding='same', activation='sigmoid')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
with strategy.scope():
  cnn_autoencoder=CNN_Autoencoder()
  cnn_autoencoder.compile(optimizer='adam',  loss='mse',steps_per_execution = 50)
  history=cnn_autoencoder.fit(X_train,X_train,epochs=20,batch_size=16,shuffle=True,validation_data=(X_test,X_test))

NameError: ignored

In [ ]:
xm_train=cnn_autoencoder.encoder(X_train)
xm_test=cnn_autoencoder.encoder(X_test)


In [ ]:
xm_train.shape

In [ ]:
xm_test.shape

In [ ]:
xm_test[1]

In [ ]:
#cnn_autoencoder.save('/gdrive/My Drive/BitirmeTezi/image_dataset_autoencoder.keras',save_format="tf")

## **Machine Learning**

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

xm_train=scaler.fit_transform(xm_train)
xm_test=scaler.transform(xm_test)

In [ ]:
from sklearn.svm import SVC

svc=SVC(kernel="rbf")
svc.fit(xm_train, y_train)

In [ ]:
y_pred=svc.predict(xm_test)

from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test, y_pred)
print(cm)


In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb=GaussianNB()
gnb.fit(xm_train, y_train)

y_pred=gnb.predict(xm_test)
cm=confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=5,metric='euclidean')

knn.fit(xm_train, y_train)
y_pred=knn.predict(xm_test)

cm=confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc=DecisionTreeClassifier(criterion="entropy")

dtc.fit(xm_train, y_train)
y_pred=dtc.predict(xm_test)

cm=confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(n_estimators=30, criterion="entropy", random_state=0)

rfc.fit(xm_train, y_train)
y_pred=rfc.predict(xm_test)

cm=confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

xm_train=scaler.fit_transform(xm_train)
xm_test=scaler.transform(xm_test)


from sklearn.naive_bayes import MultinomialNB

mnb=MultinomialNB()
mnb.fit(xm_train, y_train)

y_pred=mnb.predict(xm_test)

cm=confusion_matrix(y_test, y_pred)
print(cm)
